## If executing on Cloud9 or AWS Environment, uncomment following Code Block

In [ ]:
"""
import boto3
from sagemaker import get_execution_role

account_id = boto3.client('sts').get_caller_identity().get('Account')
ecr_repository = 'ngboost-custom-container-test'
tag = ':latest'

region = boto3.session.Session().region_name

uri_suffix = 'amazonaws.com'
if region in ['cn-north-1', 'cn-northwest-1']:
    uri_suffix = 'amazonaws.com.cn'

byoc_image_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository + tag)

byoc_image_uri
# This should return something like
# 111122223333.dkr.ecr.us-east-2.amazonaws.com/sagemaker-byoc-test:latest

role = get_execution_role()
"""

## If executing locally, uncomment following Code Block

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role

PROFILE_NAME = '' # change for your local profile on AWS cli
DEFAULT_REGION = '' # change for your region
ROLE_ARN = '' # Replace for your role arn


boto3.setup_default_session(profile_name=PROFILE_NAME) 
sess = boto3.Session(region_name=DEFAULT_REGION) 
region = sess.region_name

sts = boto3.client('sts')
account_id = sts.get_caller_identity().get('Account')
ecr_repository = 'ngboost-custom-container-test'
tag = ':latest'
uri_suffix = 'amazonaws.com'

byoc_image_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository + tag)

print(byoc_image_uri)

assumed_role_object=sts.assume_role(
    RoleArn=ROLE_ARN,
    RoleSessionName="AssumeRoleSession1"
)

credentials = assumed_role_object['Credentials']

session = boto3.Session(
    region_name=region,
    aws_access_key_id=credentials['AccessKeyId'],
    aws_secret_access_key=credentials['SecretAccessKey'],
    aws_session_token=credentials['SessionToken']
)

sagemaker_session = sagemaker.Session(boto_session=session)
role = get_execution_role(sagemaker_session=sagemaker_session)

print(role)

In [ ]:
import sagemaker
from sagemaker.estimator import Estimator

estimator = Estimator(image_uri=byoc_image_uri,
                      role=role,
                      sagemaker_session=sagemaker_session,
                      base_job_name='ngboost-custom-container-test',
                      instance_count=1,
                      instance_type='ml.m5.large')

# start training
estimator.fit()